In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnext101_32x8d
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the CUDA device if available
if torch.cuda.is_available():
    torch.cuda.set_device(0)  # Set to the desired GPU device ID
    print("set to GPU")

print(torch.version.cuda)

set to GPU
11.7


In [2]:
print(torch.__version__)

2.0.1+cu117


In [3]:
# Note: After you run this cell, the training and test data will be available in
# the file browser. (Click the folder icon on the left to view it)
#
# If you don't see the data after the cell completes, click the refresh button
# in the file browser (folder icon with circular arrow)

# First, let's download and unzip the data
#!echo "Downloading files..."
#!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/training1.zip
#!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/training2.zip
#!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/test.zip
#!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/test_partial.zip
#!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/test_classes_partial.csv

#!echo "Unzipping files..."
#!tar -xf C:\Users\dogeb\Documents\GitHub\CSE450-Team\Project_4\Spencer_Work\training1.zip
#!tar -xf C:\Users\dogeb\Documents\GitHub\CSE450-Team\Project_4\Spencer_Work\training2.zip
#!tar -xf C:\Users\dogeb\Documents\GitHub\CSE450-Team\Project_4\Spencer_Work\test.zip
#!tar -xf C:\Users\dogeb\Documents\GitHub\CSE450-Team\Project_4\Spencer_Work\test_partial.zip

# Combine the two traning directories
#!echo "Merging training data..."
#!mkdir C:\Users\dogeb\Documents\GitHub\CSE450-Team\Project_4\Spencer_Work\training
#!mv C:\Users\dogeb\Documents\GitHub\CSE450-Team\Project_4\Spencer_Work\training1\* C:\Users\dogeb\Documents\GitHub\CSE450-Team\Project_4\Spencer_Work\training
#!mv C:\Users\dogeb\Documents\GitHub\CSE450-Team\Project_4\Spencer_Work\training2\* C:\Users\dogeb\Documents\GitHub\CSE450-Team\Project_4\Spencer_Work\training

# Cleanup
#!echo "Cleaning up..."
#!rmdir C:\Users\dogeb\Documents\GitHub\CSE450-Team\Project_4\Spencer_Work\training1
#!rmdir C:\Users\dogeb\Documents\GitHub\CSE450-Team\Project_4\Spencer_Work\training2
#!rm training1.zip
#!rm training2.zip
#!rm test.zip
#!rm test_partial.zip

#!echo "Data ready."

"Downloading files..."
"Unzipping files..."
"Merging training data..."


mv: cannot stat `C:\\Users\\dogeb\\Documents\\GitHub\\CSE450-Team\\Project_4\\Spencer_Work\\training1\\*': No such file or directory
mv: cannot stat `C:\\Users\\dogeb\\Documents\\GitHub\\CSE450-Team\\Project_4\\Spencer_Work\\training2\\*': No such file or directory


"Cleaning up..."


The directory is not empty.
The directory is not empty.


"Data ready."


In [3]:
training_dir = r'C:\Users\dogeb\Documents\GitHub\CSE450-Team\Project_4\training'
image_size = (100, 100)

transform = ToTensor()

train_dataset = ImageFolder(root=training_dir, transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [4]:
class StreetSignClassifier(nn.Module):
    def __init__(self, num_classes):
        super(StreetSignClassifier, self).__init__()
        self.resnext = resnext101_32x8d(pretrained=True)
        num_filters = self.resnext.fc.in_features
        self.resnext.fc = nn.Linear(num_filters, num_classes)
    
    def forward(self, x):
        return self.resnext(x)

model = StreetSignClassifier(num_classes=43)  # Assuming there are 42 sign classes

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Calculate average training loss for the epoch
    train_loss = running_loss / len(train_dataloader)
    
    print(f"Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f}")

C:\Users\dogeb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\dogeb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt101_32X8D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt101_32X8D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
